#  🏈 Defender Bite Velocity on Play-Action 🏈

![](https://ftw.usatoday.com/wp-content/uploads/sites/90/2019/06/usatsi_11920721.jpg?w=1000&h=600&crop=1)

The play-action pass play has been a staple in NFL offenses for many years now. The play usually consists of the Quarterback faking a handoff to the runningback, before passing the ball downfield.

The main goal of the play-action is to make the defense think that you're running the ball, resulting in some defenders to be out of position on their defensive assignments, and thus allowing the offensive players to get open for a catch.


# Bite, Biting, Bit


**"Bite", "Biting", "Bit"** are common words to describe an out-of-position defender due to being deceived by the play-action.

** Examples: **

"Look how hard the Linebacker **bit** on that play-action boot play."

"They keep **biting** on the play-action, let's make sure to emphasize that in the second half."

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Objective:

For this study, we want to:

- Quantify how much a defender **"Bites"**, and what metrics might that involve
- Determine if some Positions are more deceptible than others
- Determine if some Individual Defenders are more deceptible than others
- Determine if some Quarterbacks are more effective at Play-Action than others

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Let's Get Started

![](https://media.giphy.com/media/azdYYsuaNuOoOhxwJZ/giphy.gif)

In the short clip above you will see the Titans vs. Bills week 5 matchup, in which the Titans are on offense attempting a play-action pass.

At the beginning of the play you will see Linebacker Tremaine Edmunds circled, keep your eye on him throughout the play.

Notice how Edmunds starts to move towards the ball when he sees Mariota showing a handoff, then he realizes it's a pass play and retreats back to his defensive assignment. That split second of making Edmunds commit to the run and making him out-of-position, was exactly what the Titans were hoping for.



# Measurements

So for this study we will quantify:

- **Bite Velocity**
    - Peak "Biting" Velocity of the Defender towards the line of scrimmage before they retreat to their defensive assignment
- **Bite Distance**
    - Peak "Biting" Distance relative to where they were when the ball was snapped
- **Retreat/ Get back Velocity**
    - Peak "Retreating" Velocity after they realized it was a pass

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Calculations

We will be using the same [revised data](https://www.kaggle.com/jdruzzi/revised-bdb-data), as mentioned in the [previous submission](https://www.kaggle.com/jdruzzi/shadow-cornerback-coverage-analysis). Data is already preprocessed and standardized with additonal features added on.


Bite Velocity takes into account both **Speed** and **Direction**. So how it will be measured, is by taking the player direction variable, converting it to radians, standardizing it and then we shift direction so that cosine of direction maps to the X axis, and sine of direction maps to the Y axis. Creating 2 new variables:

**dx = PlayerSpeed x cos(Standardized Direction)**

**dy = PlayerSpeed x sin(Standardized Direction)**

The assumptions are that the defender is either moving towards the line of scrimmage, or away from it. We really only want to know the players velocity relative to the X-axis, so **dx** is our measurement of choice. 

Since all data is standardized, the offense will always be moving left to right. In turn, if the defender bites on the play-action, **dx** will always be a negative value and peak bite velocity will be the largest negative **dx** value from the moment the ball is snapped until the ball leaves the QB's hands.

**Peak Bite Velocity**: Min(dx) from moment the ball is snapped until the ball leaves the QB's hands

**Peak Bite Distance**: (X Coordinate of Defender at ball snap) - Min(X) from moment the ball is snapped until the ball leaves the QB's hands

**Peak Retreat / Get Back Velocity**: Max(dx) from moment of Play Actopm until the ball leaves the QB's hands

# Criteria

In order for the Defender to be classified as "Biting" they must follow these rules:

- Peak Bite Distance must be less than 0

- The Defender's X-Coordinate when the ball is leaving the QB's hand, must be larger than their X-Coordinate at the snap of the ball. Indicating that they bit, and have retreated to their assignment.

- Peak "Get back" Velocity must be greater than 1. If we don't account for this, then at times it will appear that the defender bites tremendously, only to find out that they are actually blitzing or covering a RB/TE in the flat. We want to measure only the defenders that bite hard and retreat back towards their own endzone, as opposed to the sideline.

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 1000)
from tqdm import tqdm
from sklearn.neighbors import BallTree
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from datetime import datetime
import pytz
from IPython.display import HTML
import scipy.stats as stats
import matplotlib as mpl
from matplotlib import animation, rc, use
from matplotlib.patches import Rectangle, Arrow
import tensorflow as tf
from matplotlib.patches import Polygon
import matplotlib.patheffects as pe
import gc


def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    
    return df


def get_dx_dy(radian_angle, dist):
    dx = dist * math.cos(radian_angle)
    dy = dist * math.sin(radian_angle)
    return dx, dy


def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12*2, 6.33*2)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='slategrey', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.3,
                                edgecolor='k',
                                facecolor='royalblue',
                                alpha=0.4,
                                zorder=1)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.3,
                                edgecolor='k',
                                facecolor='royalblue',
                                alpha=0.4,
                                zorder=1)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(0, 53.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax



class CreateNFLData:

    def __init__(self):
        pass

    def LoadData(self, Normal=True):
        if Normal == True:
            print("Loading Original Data")
            globbed_files = glob.glob("week*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        
        else:
            print("Loading Modified Data")
            globbed_files = glob.glob("Revised Data/*.csv") #creates a list of all csv files
            data = []
            for csv in tqdm(globbed_files):
                frame = pd.read_csv(csv, index_col=0)
                data.append(frame)

            WeekData = pd.concat(data).reset_index()
            WeekData
        return WeekData



    def Standardize(self,W):
        print("Standardizing Data..")
        W['Dir_rad'] = np.mod(90 - W.dir, 360) * math.pi/180.0
        W['ToLeft'] = W.playDirection == "left"
        W['TeamOnOffense'] = "home"
        W.loc[W.possessionTeam != W.PlayerTeam, 'TeamOnOffense'] = "away"
        W['IsOnOffense'] = W.PlayerTeam == W.TeamOnOffense # Is player on offense?
        W['YardLine_std'] = 100 - W.yardlineNumber
        W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'YardLine_std'
                ] = W.loc[W.yardlineSide.fillna('') == W.possessionTeam,  
                'yardlineNumber']
        W['X_std'] = W.x
        W.loc[W.ToLeft, 'X_std'] = 120 - W.loc[W.ToLeft, 'x'] 
        W['Y_std'] = W.y
        W.loc[W.ToLeft, 'Y_std'] = 160/3 - W.loc[W.ToLeft, 'y'] 
        #W['Orientation_std'] = -90 + W.Orientation
        #W.loc[W.ToLeft, 'Orientation_std'] = np.mod(180 + W.loc[W.ToLeft, 'Orientation_std'], 360)
        W['Dir_std'] = W.Dir_rad
        W.loc[W.ToLeft, 'Dir_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Dir_rad'], 2*np.pi)
        W['dx'] = round(W['s']*np.cos(W['Dir_std']),2)
        W['dy'] = round(W['s']*np.sin(W['Dir_std']),2)
        W['X_std'] = round(W['X_std'],2)
        W['Y_std'] = round(W['Y_std'],2)
        #W['Orientation_rad'] = np.mod(W.o, 360) * math.pi/180.0
        W['Orientation_rad'] = np.mod(-W.o + 90, 360) * math.pi/180.0
        W['Orientation_std'] = W.Orientation_rad
        W.loc[W.ToLeft, 'Orientation_std'] = np.mod(np.pi + W.loc[W.ToLeft, 'Orientation_rad'], 2*np.pi)
        W['MPH'] = W['s'] / 0.488889
        return W


    
    def FrameData(self,WeekData1):
        NotNone = WeekData1.query('event != "None"')
        NotNone = NotNone.groupby(['gameId','playId','event'])['frameId'].max().reset_index()
        NotNone = NotNone.set_index(['gameId','playId','event'], drop= True).unstack('event').reset_index()
        NotNone.columns = [' '.join(col).strip() for col in NotNone.columns.values]
        NotNone.columns = NotNone.columns.str.replace('frameId' , '')
        NotNone.columns = NotNone.columns.str.replace(' ' , '')
        NotNone['Code'] = NotNone['gameId'].astype(str) + "-" + NotNone['playId'].astype(str)
        NotNone = NotNone.set_index('Code')
        NotNone = NotNone.loc[~NotNone.index.duplicated(keep='first')]

        for col in tqdm(NotNone.columns):
            NotNone['Contains_' + str(col)] = np.where(NotNone[col] > 0, True, False)

        Cols = ['ball_snap', 'man_in_motion', 'pass_arrived', 'pass_forward', 'play_action','pass_outcome_caught', 'run_pass_option', 'Contains_man_in_motion', 'Contains_pass_arrived', 'Contains_pass_forward', 'Contains_pass_outcome_caught', 'Contains_play_action','Contains_run_pass_option']
     #   WeekData1 = pd.merge(df, NotNone, how="left", left_on=['gameId','playId'], right_on=['gameId','playId'] )
        for col in Cols:
            WeekData1[col] = WeekData1.Code.map(NotNone[col])

        del NotNone
        gc.collect()

        WeekData1['After_snap'] = np.where(WeekData1['frameId'] > WeekData1['ball_snap'],1,0)
        WeekData1['After_Throw'] = np.where(WeekData1['frameId'] > WeekData1['pass_forward'],1,0)
        WeekData1['After_PassArrived'] = np.where(WeekData1['frameId'] > WeekData1['pass_arrived'],1,0)
        WeekData1['After_PlayAction'] = np.where(WeekData1['frameId'] > WeekData1['play_action'],1,0)
      #  WeekData1['After_run_pass_option'] = np.where(WeekData1['frameId'] > WeekData1['run_pass_option'],1,0)
        WeekData1['After_Catch'] = np.where(WeekData1['frameId'] > WeekData1['pass_outcome_caught'],1,0)


        LOS = WeekData1.query('displayName == "Football" & After_snap == 0')
        LOS = LOS.groupby(['gameId','playId'])['X_std'].agg('median').reset_index()
        LOS.columns = ['gameId','playId','LOS']
        LOS['Code'] = LOS['gameId'].astype(str) + "-" + LOS['playId'].astype(str)
        LOS = LOS.set_index('Code')
        LOS = LOS.loc[~LOS.index.duplicated(keep='first')]
        WeekData1["LOS"] = WeekData1.Code.map(LOS['LOS'])
        WeekData1['Distfrom_LOS'] = WeekData1['X_std'] - WeekData1['LOS']
        WeekData1['AbsDistfrom_LOS'] = np.abs(WeekData1['X_std'] - WeekData1['LOS'])
        del LOS
        gc.collect()
        return WeekData1

    def import_data(self,file,columns=False,cols=""):
        if columns == False:
            df = pd.read_csv(file, low_memory=False)
            df = reduce_mem_usage(df)
        else:
            df = pd.read_csv(file, low_memory=False, usecols=cols)
            df = reduce_mem_usage(df)
        return df






class AnimatePlay:
    def __init__(self, play_df,player_id, Tri = False, MPH = False,Text="",Show='jerseyNumber',method='all' ) -> None:
        self._MAX_FIELD_Y = 53.3
        self._MAX_FIELD_X = 120
        self._MAX_FIELD_PLAYERS = 22
        

        self.Tri = Tri
        self.MPH = MPH
        self.player_id = player_id
        self.Show = Show
        self.method = method
        self.Text = Text

        self._CPLT = sns.color_palette("husl", 2)
        self._frame_data = play_df
        self._times = sorted(play_df.time.unique())
        self._stream = self.data_stream()
        
        self._date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 
        self._mean_interval_ms = np.mean([delta.microseconds/1000 for delta in np.diff(np.array([pytz.timezone('US/Eastern').localize(datetime.strptime(date_string, self._date_format)) for date_string in self._times]))])
        
        self._fig, self._ax_field = create_football_field()

        self._fig.set_figheight(10)
        self._fig.set_figwidth(15)
        
        self._fig.tight_layout()
        
        self._ax_field = plt.gca()
        
        self._ax_home = self._ax_field.twinx()
        self._ax_away = self._ax_field.twinx()
        self._ax_jersey = self._ax_field.twinx()

        self.ani = animation.FuncAnimation(self._fig, self.update, frames=len(self._times), interval = self._mean_interval_ms, 
                                          init_func=self.setup_plot, blit=False)
        
        plt.close()
       
    @staticmethod
    def set_axis_plots(ax, max_x, max_y) -> None:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        ax.set_xlim([0, max_x])
        ax.set_ylim([0, max_y])
        
    @staticmethod
    def convert_orientation(x):
        return (-x + 90)%360
    
    @staticmethod
    def polar_to_z(r, theta):
        return r * np.exp( 1j * theta)
    
    @staticmethod
    def deg_to_rad(deg):
        return deg*np.pi/180
        
    def data_stream(self):
        for time in self._times:
            yield self._frame_data[self._frame_data.time == time]
    
    def setup_plot(self): 
        self.set_axis_plots(self._ax_field, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        ball_snap_df = self._frame_data[(self._frame_data.event == 'ball_snap') & (self._frame_data.team == 'football')]
        self._ax_field.axvline(ball_snap_df.X_std.to_numpy()[0], color = 'yellow', linestyle = '--')
        
        self.set_axis_plots(self._ax_home, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_away, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_jersey, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        for idx in range(10,120,10):
            self._ax_field.axvline(idx, color = 'k', linestyle = '-', alpha = 0.05)
            
        self._scat_field = self._ax_field.scatter([], [], s = 200, color = 'red')
        self._scat_home = self._ax_home.scatter([], [], s = 900, color = self._CPLT[0], edgecolors = 'k')
        self._scat_away = self._ax_away.scatter([], [], s = 900, color = self._CPLT[1], edgecolors = 'k')
        
        self._scat_jersey_list = []
        self._scat_number_list = []
        self._scat_name_list = []
        self._scat_mph_list = []
        self._a_dir_list = []
        self._a_or_list = []
        self._a_tri_list = []
        for _ in range(self._MAX_FIELD_PLAYERS):
            self._scat_jersey_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black',fontweight='bold',fontsize='large',path_effects=[pe.withStroke(linewidth=3, foreground="white")]))
            self._scat_number_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'white',fontweight='bold',fontsize=14,path_effects=[pe.withStroke(linewidth=5, foreground="dodgerblue")]))
            self._scat_name_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black',fontweight='bold',fontsize='larger',path_effects=[pe.withStroke(linewidth=5, foreground="gold")]))
            self._scat_mph_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'lime',fontweight='bold',fontsize='larger'))

            self._a_dir_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            self._a_or_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            self._a_tri_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            
        return (self._scat_field, self._scat_home, self._scat_away,*self._scat_mph_list, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
        
    def update(self, anim_frame):
        pos_df = next(self._stream)
        
        for label in pos_df.team.unique():
            label_data = pos_df[pos_df.team == label]

            if label == 'football':
                self._scat_field.set_offsets(np.hstack([label_data.X_std, label_data.Y_std]))
            elif label == 'home':
                self._scat_home.set_offsets(np.vstack([label_data.X_std, label_data.Y_std]).T)
            elif label == 'away':
                self._scat_away.set_offsets(np.vstack([label_data.X_std, label_data.Y_std]).T)

        jersey_df = pos_df[pos_df.jerseyNumber.notnull()]
        
        for (index, row) in pos_df[pos_df.jerseyNumber.notnull()].reset_index().iterrows():
            self._scat_jersey_list[index].set_position((row.X_std, row.Y_std))
            self._scat_jersey_list[index].set_text(row.position)
            if self.method == 'single':
                self._scat_number_list[index].set_text(np.where(self.player_id == row.nflId,str(self.Text) +" "+ str(round(row[self.Show],2)),""))
                self._scat_number_list[index].set_position((row.X_std, row.Y_std+2.4))
            else:
                self._scat_number_list[index].set_text((row[self.Show]))
                self._scat_number_list[index].set_position((row.X_std, row.Y_std+2.4))
            self._scat_name_list[index].set_text(np.where(row.frameId <= 10,row.displayName.split()[-1],""))
            self._scat_name_list[index].set_position((row.X_std, row.Y_std-1.9))
            if self.MPH == True:
                self._scat_mph_list[index].set_text(np.where((row.s / 0.488889) > 17,str(round(float(row.s / 0.488889),2)) + " MPH",""))
                self._scat_mph_list[index].set_position((row.X_std, row.Y_std+1.9))
            else:
                pass

            player_vel = np.array([row.dx, row.dy])
            player_orient = np.array([np.real(self.polar_to_z(3, row.Orientation_std)), np.imag(self.polar_to_z(3, row.Orientation_std))])
            
            self._a_dir_list[index].remove()
            self._a_dir_list[index] = self._ax_field.add_patch(Arrow(row.X_std, row.Y_std, player_vel[0], player_vel[1], color = 'black'))
            
            self._a_or_list[index].remove()
            self._a_or_list[index] = self._ax_field.add_patch(Arrow(row.X_std, row.Y_std, player_orient[0], player_orient[1], color = 'blue', width = 2))

            if self.Tri == True:
                if self.method == 'single':
                    self._a_tri_list[index].remove()
                    self._a_tri_list[index] = self._ax_field.add_patch(Polygon([[row.X_std, row.Y_std], [row.X_std_COpp,row.Y_std_COpp],[row.X_std_QB,row.Y_std_QB]], closed=True, fill=False, hatch='/',color='lime'))
                else:
                    self._a_tri_list[index].remove()
                    self._a_tri_list[index] = self._ax_field.add_patch(Polygon([[row.X_std, row.Y_std], [row.X_std_COpp,row.Y_std_COpp],[row.X_std_QB,row.Y_std_QB]], closed=True, fill=False, hatch='/',color='lime'))
            else:
                pass
        
        return (self._scat_field, self._scat_home, self._scat_away, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)


In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 1000)

from sklearn.neighbors import BallTree


from IPython.core.display import HTML
import time
import math
from scipy.spatial import Voronoi, voronoi_plot_2d
from datetime import datetime
import pytz
from IPython.display import HTML
import scipy.stats as stats
import matplotlib as mpl
from matplotlib import animation, rc
from matplotlib.patches import Rectangle, Arrow
import tensorflow as tf

import glob
import os

import gc

def convert_orientation(x):
    return (x)%360

def deg_to_rad(deg):
        return deg*np.pi/180



Create = CreateNFLData()

start = time.process_time()

Weeks = range(1,18)

#globbed_files = glob.glob("../input/revised-data/*.csv") #creates a list of all csv files
data = []
for n in tqdm(Weeks):
    filename = '../input/revised-data/week' + str(n) + '.csv'
    frame = Create.import_data(filename,columns=True,cols=['week', 'gameId', 'playId', 'frameId', 'time', 'nflId', 'displayName', 'jerseyNumber', 'position', 'team', 'X_std', 'Y_std', 'Dir_std', 'dx', 'dy', 'Orientation_std', 's', 'MPH', 'a', 'dis', 'event', 'PlayerTeam', 'yardlineNumber', 'YardLine_std', 'OnOffense', 'TeamOnOffense', 'IsOnOffense', 'closestQB_Id', 'FootDist', 'Targeted'])
    frame['Code'] = frame['gameId'].astype(str) + "-" + frame['playId'].astype(str)
    plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv', usecols=['gameId', 'playId','down', 'yardsToGo','penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI','offenseFormation',	'personnelO',	'defendersInTheBox',	'numberOfPassRushers',	'personnelD',	'typeDropback','playType'])
    plays['Code'] = plays['gameId'].astype(str) + "-" + plays['playId'].astype(str)
    plays = plays.set_index('Code')
    plays = plays.loc[~plays.index.duplicated(keep='first')]
    Cols = ['playId','down', 'yardsToGo','penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI','offenseFormation',	'personnelO',	'defendersInTheBox',	'numberOfPassRushers',	'personnelD',	'typeDropback','playType']
    for col in Cols:
        frame[col] = frame.Code.map(plays[col])
    data.append(frame)
    del plays
    gc.collect()
    del frame
    gc.collect()

WeekData = pd.concat(data).reset_index()
del data
gc.collect()
Finaldf1 = Create.FrameData(WeekData)
del WeekData
gc.collect()
Finaldf1.drop(['index','Code','IsOnOffense', 'ball_snap', 'man_in_motion', 'pass_arrived', 'pass_forward', 'pass_outcome_caught', 'play_action', 'run_pass_option'], axis=1, inplace=True)
print(Finaldf1.memory_usage().sum() / (1024**2))
Obs = Finaldf1.select_dtypes(include=['object']).drop(['time','displayName','position'], axis=1).columns.tolist()
Finaldf1[Obs] = Finaldf1[Obs].astype('category')
del Obs
gc.collect()
print(Finaldf1.memory_usage().sum() / (1024**2))

In [ ]:
import sys
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

Pos = ['OLB','ILB','LB','MLB','FS','SS','S','DB']

Events = ['ball_snap','play_action','run_pass_option','pass_forward','pass_arrived','None']


df = Finaldf1[(Finaldf1['position'].isin(Pos)) & (Finaldf1['Contains_play_action'] == True) & (Finaldf1['event'].isin(Events)) & (Finaldf1['playType'] == "play_type_pass") ]
df['Event2'] = np.where(df['event'] == "ball_snap","ball_snap","Before_snap")
df['Event2'] = np.where((df['After_snap'] == 1) & (df['After_PlayAction'] == 0),"AfterSnap_BeforePA",df['Event2'])
df['Event2'] = np.where(df['event'] == "play_action","play_action",df['Event2'])
df['Event2'] = np.where((df['After_PlayAction'] == 1) & (df['After_Throw'] == 0),"AfterPA_BeforeThrow",df['Event2'])
df['Event2'] = np.where(df['event'] == "pass_forward","pass_forward",df['Event2'])
df['Event2'] = np.where((df['After_Throw'] == 1) & (df['After_PassArrived'] == 0),"Ball in the Air", df['Event2'])
df['Event2'] = np.where(df['event'] == "pass_arrived","pass_arrives",df['Event2'])
df['Event2'] = np.where((df['After_Throw'] == 1) & (df['After_PassArrived'] == 1),"After Catch", df['Event2'])

df['EventCount'] = df.groupby(['gameId','playId','Event2'])['Event2'].transform('count') / df.groupby(['gameId','playId'])['nflId'].transform('nunique')
df['EventOrder'] = df.groupby(['gameId','playId','Event2'])['frameId'].rank(ascending=True, method='dense').astype(int)
df['EventPct'] = df['EventOrder'] / df['EventCount']

df['Partition'] = np.where(df['EventPct'] > (1/2), "2nd Phase", "1st Phase")

df['Group'] = df['Event2'] + "-" + df['Partition']

Events = df.groupby(['gameId','playId','nflId','displayName','position','closestQB_Id','Event2'])['Distfrom_LOS','dx'].agg({'Distfrom_LOS':[('min', 'min')],'dx':[('min', 'min'),('max', 'max') ],}).reset_index(drop=False)

del df
gc.collect()


Events.columns = [' '.join(col).strip() for col in Events.columns.values]
cols = [ 'Distfrom_LOS min','dx min','dx max' ]
Events[cols] = Events[cols].applymap(np.float32)
Events = Events.set_index(['gameId','playId','nflId','displayName','position','closestQB_Id','Event2'], drop= True).unstack('Event2').reset_index()
Events.columns = [' '.join(col).strip() for col in Events.columns.values]
#Events.columns = Events.columns.str.replace('min' , '')
#Events.columns = Events.columns.str.replace('max' , '')
Events.columns = Events.columns.str.replace(' ' , '_')
Events = Events[Events.columns.drop(list(Events.filter(regex='After_Catch')))]
Events = Events[Events.columns.drop(list(Events.filter(regex='Before_snap')))]
Events = Events[Events.columns.drop(list(Events.filter(regex='Ball_in_the_Air')))]
Events = Events[Events.columns.drop(list(Events.filter(regex='pass_arrives')))]
Cols = ['gameId','playId','nflId','displayName','position','closestQB_Id', 'Distfrom_LOS_min_ball_snap',  'Distfrom_LOS_min_AfterSnap_BeforePA', 'Distfrom_LOS_min_play_action', 'Distfrom_LOS_min_AfterPA_BeforeThrow','Distfrom_LOS_min_pass_forward','dx_min_ball_snap','dx_min_AfterSnap_BeforePA', 'dx_max_AfterSnap_BeforePA','dx_min_play_action', 'dx_min_AfterPA_BeforeThrow', 'dx_max_AfterPA_BeforeThrow']
Events = Events[Cols]
#Events['Bite_Distance'] = Events.iloc[:,-4:].min(axis = 1, skipna = True) - Events['Distfrom_LOSball_snap']
Events['Bite_Distance'] = Events.iloc[:,-12:-8].min(axis = 1, skipna = True) - Events['Distfrom_LOS_min_ball_snap']
Events['Bite_Velocity'] = Events.iloc[:,-7:-3].min(axis = 1, skipna = True)
Events['Getback_Velocity'] = Events.iloc[:,-7:-2].max(axis = 1, skipna = True)
Events['Bite'] = np.where((Events['Distfrom_LOS_min_pass_forward'] > Events['Distfrom_LOS_min_ball_snap']) & (Events['Bite_Distance'] < 0) & (Events['Getback_Velocity'] > 1),1,0)
Events['Pos2'] = np.where(Events['position'].str.contains("LB"), "LB","Safety")
gc.collect()
Events

# First Look

Thanks to tracking data, we can now analyze this type of information! Below you will find the same play as the clip above, except now using the data associated with it

Please note "dx" atop Edmund's player icon. Now you can see in realtime how his velocity changes throughout the play. 

Blue Arrows = The direction the player is facing

Black Arrows = The direction the player is moving

In [ ]:
from matplotlib.patches import Polygon
import matplotlib.patheffects as pe
#from BDBUtils.Utilities import AnimatePlay

gId = 2018100700	
pId = 1799
nflId = 2560713.0

ex = Finaldf1.query('gameId == @gId & playId == @pId & nflId == @nflId | gameId == @gId & playId == @pId & OnOffense == True | gameId == @gId & playId == @pId & displayName == "Football"')
ex1 = Events.query('gameId == @gId & playId == @pId & nflId == @nflId ').filter(['gameId','playId','displayName','position','Bite_Distance','Bite_Velocity','Getback_Velocity','Bite'],axis=1)

display(HTML(ex1.to_html()))
animated_play = AnimatePlay(ex,nflId,Tri = False, MPH = True,Text="dx:",Show='dx',method='single')
HTML(animated_play.ani.to_jshtml())

You can see that Edmund's hit his Peak Bite Velocity around -2.78, before realizing that it is a pass play, then retreats and hits a max Get Back velocity of 6.44.

## Tremaine Edmunds

### Did he bite? Yes

### Peak Bite Distance = -2.625

### Peak Bite Velocity = -2.779297

### Peak Retreat / Get Back Velocity = 6.441406

![](http://)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Bite Percent - by Player

Generally the only positions that bite on play-action are Linebackers and Safeties, Corners and Defensive Linemen wil be left out of this analysis.

*Criteria: Min of 15 Play Action snaps*

In [ ]:
Group = Events.groupby(['nflId','displayName','position','Pos2'])['Bite'].agg(['count','sum','mean']).reset_index()
Cols = ['nflId', 'displayName', 'position', 'Pos2', 'count','sum','mean']
Group = Group[Cols]
Group.columns = ['nflId', 'displayName', 'position', 'Pos2', 'PlayAction_snaps', 'Number_of_Bites', 'BitePct']
Group = Group.query('PlayAction_snaps > 15', engine='python').sort_values(by=['BitePct'], ascending=False)#.head(10)
Group = Group.query('position != "S" & position != "DB"').sort_values(by=['BitePct'], ascending=False).reset_index()
HighTen = Group.head(10)
LowTen = Group.tail(10)

print("Top 10")
display(HTML(HighTen.to_html()))

import plotly.express as px

fig = px.bar(HighTen, x='displayName', y='BitePct',color="position",template='plotly_dark',title="Average Bite % On Play Action -  By Player",color_discrete_sequence=px.colors.qualitative.T10)
fig.update_layout(
    autosize=False,
    width=800,
    height=650,
    yaxis=dict(title_text="Bite Percent"),
    xaxis=dict(title_text="Player")
)
fig.show()

Kyle Emanuel bit on around 74% of play-action snaps in 2018

# Bite Percent - by Position

Bite percent was calculated by taking the average from 20 trials of 12 random samples for each position group with replacement.

In [ ]:
Alldfs = []

rounds = range(1,20)

for n in rounds:
    OLB = Group.query('position == "OLB"').sample(n=12)
    FS = Group.query('position == "FS"').sample(n=12)
    SS = Group.query('position == "SS"').sample(n=12)
    ILB = Group.query('position == "ILB"').sample(n=12)
    LB = Group.query('position == "LB"').sample(n=12)
    MLB = Group.query('position == "MLB"').sample(n=12)
    Plotdf = pd.concat([OLB,FS,SS,ILB,LB,MLB])
    Alldfs.append(Plotdf)

#Plotdf = pd.concat([OLB,FS,SS,ILB,LB,MLB])
Plotdf = pd.concat(Alldfs)
Plotdf = Plotdf.groupby(['position','Pos2'])['BitePct'].agg(['mean']).reset_index()

fig = px.bar(Plotdf, x='position', y='mean',color="Pos2",template='plotly_dark',title="Average Bite % On Play Action -  By Position")
fig.update_layout(
    autosize=False,
    width=800,
    height=650,
    yaxis=dict(title_text="Bite Percent"),
    xaxis=dict(title_text="Position")
)
fig.show()

Middle Linebackers generally tend to bite more on play-action, and Linebackers in general appear to bite more than Safeties.

# Bite Velocity - by Player

This tells us which players usually tend to over-commit to the run on play-action

In [ ]:
Bites = Events.query('Bite == 1 & Bite_Distance < 0', engine='python')
Bites = Bites.groupby(['nflId','displayName','position'])['Bite_Distance','Bite_Velocity','Getback_Velocity'].agg(['mean','count']).reset_index()
Bites.columns = [' '.join(col).strip() for col in Bites.columns.values]
Bites.columns = Bites.columns.str.replace(' ' , '_')
Bites.drop(['Bite_Distance_count','Getback_Velocity_count'], axis=1, inplace=True)
Bites = Bites.query('Bite_Velocity_count > 20').sort_values(by=['Bite_Velocity_mean'], ascending=True)
HighTen = Bites.head(10)


print("Top 10")
display(HTML(HighTen.to_html()))

import plotly.express as px

fig = px.bar(HighTen, x='displayName', y=np.abs(HighTen['Bite_Velocity_mean']),color="position",template='plotly_dark',title="Average Absolute Bite Velocity - By Player",barmode='relative',color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(
    autosize=False,
    width=800,
    height=650,
    yaxis=dict(title_text="Bite Velocity"),
    xaxis=dict(title_text="Player")
)

fig.show()

You can see Tremaine Edmunds from our example, is also one of the ones that has a high Bite Velocity

# Bite Percent vs. Bite Velocity - by Quarterback

This tells us which Quarterbacks are generally more effective at luring a defender in on their play-action play

*Min 100 play-action snaps*

In [ ]:
Bites = Events.query('Bite == 1')

QBBiteVelocity = Bites.groupby(['closestQB_Id'])['Bite_Velocity'].agg(['count','mean']).reset_index()


players = pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv', usecols=['nflId','displayName'])
players = players.set_index('nflId')
players = players.loc[~players.index.duplicated(keep='first')]
QBBiteVelocity["Name"] = QBBiteVelocity.closestQB_Id.map(players['displayName'])
#QBBiteVelocity = QBBiteVelocity.query('count > 100').sort_values(by=['mean'], ascending=True)
QBBiteVelocity = QBBiteVelocity.set_index('closestQB_Id')
QBBiteVelocity = QBBiteVelocity.loc[~QBBiteVelocity.index.duplicated(keep='first')]

QBs = Events.groupby(['closestQB_Id'])['Bite'].agg(['count','sum','mean']).reset_index()


players = pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv', usecols=['nflId','displayName'])
players = players.set_index('nflId')
players = players.loc[~players.index.duplicated(keep='first')]
QBs["Name"] = QBs.closestQB_Id.map(players['displayName'])
QBs["Velocity"] = QBs.closestQB_Id.map(QBBiteVelocity['mean'])
QBs = QBs.query('count > 100').sort_values(by=['mean'], ascending=False)

del players
gc.collect()

fig = px.scatter(QBs, x=np.abs(QBs['mean']), y=np.abs(QBs['Velocity']),color="Name",template='plotly_dark',title="Bite Percent vs. Bite Velocity - By Quarterback",color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_layout(
    autosize=False,
    width=800,
    height=650,
    yaxis=dict(title_text="Avg. Bite Velocity"),
    xaxis=dict(title_text="Avg. Bite Percent")
)

fig.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Final Thoughts:

We've seen that we can quantify, and show defenders that are more susceptible to the play-action, we've also seen that some Quarterbacks were more successful than others at deceiving the defense. I think this can definitely lay the ground work for many possible studies, regarding how to optimize the play-action. 


# Who will find this submission useful:

- NFL Analytics crew using this method to isolate defenders that are easily deceived, also delve into this work further by determining what characteristics make-up an effective play-action. 

- Inspiring Fantasy sites / private advanced NFL stat companies to create new metrics around this

- A curious NFL fan

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Other Work:

1. [Shadow Cornerback + Coverage Analysis](https://www.kaggle.com/jdruzzi/shadow-cornerback-coverage-analysis)

2. [Defender Bite Velocity on Play-Action](https://www.kaggle.com/jdruzzi/defender-bite-velocity-on-play-action)

3. [Pass Coverage Classification](https://www.kaggle.com/jdruzzi/pass-coverage-classification-80-recall)

4. [Quantifying Press Coverage](https://www.kaggle.com/jdruzzi/quantifying-press-coverage-ability)

5. [Defender Tendencies: One-Cut Routes + Double Moves](https://www.kaggle.com/jdruzzi/defender-tendencies-one-cut-routes-double-moves)

## Data:

[Revised BDB Data](https://www.kaggle.com/jdruzzi/revised-bdb-data)
